In [3]:
import csv
import os
import pickle
from tqdm import tqdm

In [4]:
subdirs = ['Training','Development','Testing']
dir_prefix = os.path.join(os.getcwd(), 'UDData')

In [25]:
with open('UDData/Development/hi_hdtb-ud-dev.conllu.txt', 'r') as f:
    t = f.read()
    t = t.split('\n')
    print(t[0].split('\t'), len(t[0].split('\t')))
    print(t[1].split('\t'), len(t[1].split('\t')))
    print(t[2].split('\t'), len(t[2].split('\t')))
    print(t[26].split('\t'), len(t[26].split('\t')))
    print(t[26].split('\t')[0] == '')
    print(t[3])
    print(t[3].split('\t'))
    m = t[3].split('\t')
    print(m[1], m[4], m[5], m[7], m[9])
    print(m[5]+'|'+m[9])
    lengths = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    for line in t:
        line = line.split('\t')
        lengths[len(line)] += 1
    print(lengths)
    text = m[5]+'|'+m[9]
    print(text.split('|'))
    print(text.split('|')[0].split('=')[1])

['# sent_id = dev-s1'] 1
['# text = रामायण काल में भगवान राम के पुत्र कुश की राजधानी कुशावती को 483 ईसा पूर्व बुद्ध ने अपने अंतिम विश्राम के लिए चुना ।'] 1
['1', 'रामायण', 'रामायण', 'PROPN', 'NNPC', 'Case=Nom|Gender=Masc|Number=Sing|Person=3', '2', 'compound', '_', 'Vib=0|Tam=0|ChunkId=NP|ChunkType=child|Translit=rāmāyaṇa'] 10
[''] 1
True
2	काल	काल	PROPN	NNP	Case=Acc|Gender=Masc|Number=Sing|Person=3	23	obl	_	Vib=0_में|Tam=0|ChunkId=NP|ChunkType=head|Translit=kāla
['2', 'काल', 'काल', 'PROPN', 'NNP', 'Case=Acc|Gender=Masc|Number=Sing|Person=3', '23', 'obl', '_', 'Vib=0_में|Tam=0|ChunkId=NP|ChunkType=head|Translit=kāla']
काल NNP Case=Acc|Gender=Masc|Number=Sing|Person=3 obl Vib=0_में|Tam=0|ChunkId=NP|ChunkType=head|Translit=kāla
Case=Acc|Gender=Masc|Number=Sing|Person=3|Vib=0_में|Tam=0|ChunkId=NP|ChunkType=head|Translit=kāla
[0, 4978, 0, 0, 0, 0, 0, 0, 0, 0, 35217, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
['Case=Acc', 'Gender=Masc', 'Number=Sing', 'Person=3', 'Vib=0_में', 'Tam=0', 'Ch

In [26]:
def get_features(pos, featurelist, word):
    featuredict = {}
    featuredict['pos'] = pos
    featuredict['word'] = word
    featuresplit = featurelist.split('|')
    for feature in featuresplit:
        if '=' in feature:
#             print(feature)
            key = feature.split('=')[0]
            val = feature.split('=')[1]
#         if not (key == 'stype' or key == 'voicetype' or key == 'chunkType' or key == 'chunkId'):
            featuredict[key] = val
#         else:
#             print(feature)
    return featuredict
    
for current in subdirs:
    all_sentences = []
    all_features = []
    all_ud_tags = []
    
    directory = os.path.join(dir_prefix, current)
    print("processing files in directory: ", directory)
    for file in os.listdir(directory):
        file = os.path.join(directory, file)
        with open(file) as readfile:
            filerows = readfile.read().split('\n')
            
            sentence = []
            ud_tags = []
            features = []
            
            for row in tqdm(filerows):
                row = row.split('\t')
            
                if len(row) == 1 and row[0] == '':
                    all_sentences.append(sentence)
                    all_features.append(features)
                    all_ud_tags.append(ud_tags)
                    
                    sentence = []
                    features = []
                    ud_tags = []
                elif len(row) == 1:
                    continue
                else:
                    try:
                        sentence.append(row[1])
                        features.append(get_features(row[4], row[5]+'|'+row[9], row[1]))
                        ud_tags.append(row[7])
                    except:
                        print("Error with sentence in file:", file, ", no need to interfere")
    datadir = os.path.join(os.getcwd(), 'Data/')                    
    with open(datadir+current+'_sentences.pkl', 'wb') as dump:
        pickle.dump(all_sentences, dump)
    with open(datadir+current+'_features.pkl', 'wb') as dump:
        pickle.dump(all_features, dump)
    with open(datadir+current+'_ud_tags.pkl', 'wb') as dump:
        pickle.dump(all_ud_tags, dump)
        
    if current == 'Testing':
        testcases = {}
        testcases['sentences'] = all_sentences
        testcases['ud'] = all_ud_tags
        testcases['features'] = all_features
        with open('testfile.pkl', 'wb') as testfile:
            pickle.dump(testcases, testfile)
        print('testfile written')

processing files in directory:  /home/zubair/Documents/Work/Acads/Semester6/subjects/LData/Assignment/Project/Code/udmodules/UDData/Training


 35%|███▌      | 14085/40195 [00:00<00:00, 140849.13it/s]

processing files in directory:  /home/zubair/Documents/Work/Acads/Semester6/subjects/LData/Assignment/Project/Code/udmodules/UDData/Development


 22%|██▏       | 8957/40483 [00:00<00:00, 89569.66it/s]

processing files in directory:  /home/zubair/Documents/Work/Acads/Semester6/subjects/LData/Assignment/Project/Code/udmodules/UDData/Testing


100%|██████████| 40483/40483 [00:00<00:00, 96183.17it/s]


testfile written


## Testing some data

In [45]:
# with open('AnnCorra/Training/fullnews_id_2575975_date_7_7_2004.dat', newline='') as csvfile:
#     csvreader = csv.reader(csvfile, delimiter='\t')
#     for row in csvreader:
#         if len(row) > 0:
#             print(row[1], row[4], row[5].split('|'), row[7])

In [27]:
with open('Data/Testing_features.pkl', 'rb') as pickled:
    newlist = pickle.load(pickled)
print(len(newlist))
print(newlist[2])

1685
[{'pos': 'NNP', 'word': 'भैरव', 'Case': 'Acc', 'Gender': 'Masc', 'Number': 'Sing', 'Person': '3', 'Vib': '0_का', 'Tam': '0', 'ChunkId': 'NP', 'ChunkType': 'head', 'Translit': 'bhairava'}, {'pos': 'PSP', 'word': 'का', 'AdpType': 'Post', 'Case': 'Nom', 'Gender': 'Masc', 'Number': 'Sing', 'ChunkId': 'NP', 'ChunkType': 'child', 'Translit': 'kā'}, {'pos': 'NN', 'word': 'स्थान', 'Case': 'Nom', 'Gender': 'Masc', 'Number': 'Sing', 'Person': '3', 'Vib': '0', 'Tam': '0', 'ChunkId': 'NP2', 'ChunkType': 'head', 'Translit': 'sthāna'}, {'pos': 'NNP', 'word': 'उत्तराखंड', 'Case': 'Acc', 'Gender': 'Masc', 'Number': 'Sing', 'Person': '3', 'Vib': '0_में', 'Tam': '0', 'ChunkId': 'NP3', 'ChunkType': 'head', 'Translit': 'uttarākhaṁḍa'}, {'pos': 'PSP', 'word': 'में', 'AdpType': 'Post', 'ChunkId': 'NP3', 'ChunkType': 'child', 'Translit': 'meṁ'}, {'pos': 'NNP', 'word': 'क्षेत्रपाल', 'Case': 'Acc', 'Gender': 'Masc', 'Number': 'Sing', 'Person': '3', 'Vib': '0', 'Tam': '0', 'ChunkId': 'NP4', 'ChunkType': 'h